In [1]:
from datasets import load_from_disk, Dataset
from tqdm import tqdm, trange
import pandas as pd
import pickle
import re

In [2]:
# doc-id - context
with open("/opt/ml/data/wiki_id_context_pair.bin", "rb") as f:
    wiki_id_context = pickle.load(f)
# context - doc_id
with open("/opt/ml/data/wiki_context_id_pair.bin", "rb") as f:
    wiki_context_id = pickle.load(f)
# retrieval한 비슷한 query - doc id dictionary
with open('/opt/ml/data/elastic_train_1000.bin', "rb") as file:
    trian_query_ids = pickle.load(file)
# with open('/opt/ml/data/hybrid_train_retrieval_1000.bin', "rb") as file:
#     trian_query_ids = pickle.load(file)

In [3]:
def preprocess(text):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\\n', ' ', text) # remove newline character
    text = re.sub(r'\s+', ' ', text) # remove continuous spaces
    text = re.sub(r'#', ' ', text)
    return text
#나중 실험을 위한...

In [4]:
train_dataset = load_from_disk("/opt/ml/data/new_train_dataset/train").to_pandas() # 전처리된 데이터를 테스트로 활용
origin_train_dataset = load_from_disk("/opt/ml/data/train_dataset/train").to_pandas()

In [5]:
wiki_ids = []

for i in range(len(train_dataset)):
    ctx = train_dataset['context'][i] # context
    wiki_ids.append(wiki_context_id[ctx]) # 미리 저장해둔 dict으로 wiki doc id를 찾음

train_dataset['doc_id'] = wiki_ids


In [6]:
train_in_wiki_ids = train_dataset['doc_id'].values.tolist()
train_in_wiki_ids = {k:i for i, k in enumerate(train_in_wiki_ids)}

In [7]:
new_context = []

for i in tqdm(range(len(train_dataset))):

    train_context = train_dataset['context'][i] # ground truth
    
    query = train_dataset['question'][i] # query
    ctx_wiki_ids = trian_query_ids[query] # hybrid[query] = doc_ids
    answer = train_dataset['answers'][i]['text'][0]
    
    cnt = 4 # 추가할 갯수
    #sim_top_k = [origin_train_dataset['context'][i]]
    sim_top_k = [train_context]

    pre_ground = train_context
    # pre_ground = preprocess(train_context) # 전처리 안됬을 경우
    # sim_top_k = [pre_ground]

    idx = 0

    while cnt != 0:
        concat_cxt = wiki_id_context[ctx_wiki_ids[idx]] # id 를 cxt로 변환

        if pre_ground != concat_cxt and not (answer in concat_cxt) and not (ctx_wiki_ids[idx] in train_in_wiki_ids):
            # 비슷한 context를 추가하되 정답을 포함하지 않는 문장을 추가한다.
            sim_top_k.append(concat_cxt)
            cnt -= 1
            train_in_wiki_ids[ctx_wiki_ids[idx]] = 0 #최대한 많은 passage를 보게하기 위해
        idx += 1
        if idx == 1000: # index를 넘어가면 break
            break
    add_sim_context = ' '.join(sim_top_k)
    new_context.append(add_sim_context)

100%|██████████| 3952/3952 [00:00<00:00, 20596.10it/s]


In [8]:
len(train_in_wiki_ids.keys())

19148

In [9]:
train_df = load_from_disk("/opt/ml/data/new_train_dataset/train").to_pandas()
#train_df = origin_train_dataset
train_df['context'] = new_context

In [10]:
# wiki_ids = []

# for i in range(len(train_dataset)):
#     ctx = train_dataset['context'][i] # context
#     wiki_ids.append(wiki_context_id[ctx]) # 미리 저장해둔 dict으로 wiki doc id를 찾음

# train_dataset['doc_id'] = wiki_ids

# train_in_wiki_ids = train_dataset['doc_id'].values.tolist()
# train_in_wiki_ids = {k:i for i, k in enumerate(train_in_wiki_ids)}


new_context = []

for i in tqdm(range(len(train_dataset))):

    train_context = train_dataset['context'][i] # ground truth
    
    query = train_dataset['question'][i] # query
    ctx_wiki_ids = trian_query_ids[query] # hybrid[query] = doc_ids
    answer = train_dataset['answers'][i]['text'][0]
    
    cnt = 4 # 추가할 갯수
    sim_top_k = [origin_train_dataset['context'][i]]
    #sim_top_k = [train_context]

    pre_ground = train_context
    # pre_ground = preprocess(train_context) # 전처리 안됬을 경우
    # sim_top_k = [pre_ground]

    idx = 0

    while cnt != 0:
        concat_cxt = wiki_id_context[ctx_wiki_ids[idx]] # id 를 cxt로 변환

        if pre_ground != concat_cxt and not (answer in concat_cxt) and not (ctx_wiki_ids[idx] in train_in_wiki_ids):
            # 비슷한 context를 추가하되 정답을 포함하지 않는 문장을 추가한다.
            sim_top_k.append(concat_cxt)
            cnt -= 1
            train_in_wiki_ids[ctx_wiki_ids[idx]] = 0 #최대한 많은 passage를 보게하기 위해
        idx += 1
        if idx == 1000: # index를 넘어가면 break
            break
    add_sim_context = ' '.join(sim_top_k)
    new_context.append(add_sim_context)


100%|██████████| 3952/3952 [00:00<00:00, 10087.73it/s]


In [12]:
len(train_in_wiki_ids.keys())

34956

In [13]:
train_origin_df = load_from_disk("/opt/ml/data/train_dataset/train").to_pandas()
#train_df = origin_train_dataset
train_origin_df['context'] = new_context
train_origin_df = train_origin_df[train_df.columns]

In [14]:
train_df.head()

,answers,context,id,question
0,"{'answer_start': [229], 'text': ['하원']}",미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,mrc-1-000067,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?
1,"{'answer_start': [212], 'text': ['《경영의 실제》']}",'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,mrc-0-004397,현대적 인사조직관리의 시발점이 된 책은?
2,"{'answer_start': [503], 'text': ['백성']}",강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,mrc-1-000362,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?
3,"{'answer_start': [615], 'text': ['중국']}","불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",mrc-0-001510,11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?
4,"{'answer_start': [30], 'text': ['4개']}",동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,mrc-0-000823,명문이 적힌 유물을 구성하는 그릇의 총 개수는?


In [15]:
train_origin_df.head()

,answers,context,id,question
0,"{'answer_start': [235], 'text': ['하원']}",미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,mrc-1-000067,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?
1,"{'answer_start': [212], 'text': ['《경영의 실제》']}",'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,mrc-0-004397,현대적 인사조직관리의 시발점이 된 책은?
2,"{'answer_start': [510], 'text': ['백성']}",강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,mrc-1-000362,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?
3,"{'answer_start': [625], 'text': ['중국']}","불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",mrc-0-001510,11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?
4,"{'answer_start': [30], 'text': ['4개']}",동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,mrc-0-000823,명문이 적힌 유물을 구성하는 그릇의 총 개수는?


In [16]:
# save datasets
new_train_dataset = Dataset.from_pandas(train_df)
#new_train_dataset.save_to_disk("/opt/ml/data/new_train_dataset/train_pre_es_no_dup_wiki")
print('done')

done


In [17]:
origin_train_dataset = Dataset.from_pandas(train_origin_df)
#origin_train_dataset.save_to_disk("/opt/ml/data/new_train_dataset/train_ori_es_no_dup_wiki")
print('done')

done


In [22]:
concat_train = pd.concat([train_df,train_origin_df],ignore_index=True)
concat_train_dataset = Dataset.from_pandas(concat_train)
concat_train_dataset.save_to_disk("/opt/ml/data/new_train_dataset/train_concat_es_no_dup_more_wiki")
print('done')

done


In [23]:
len(concat_train)

7904

In [21]:
len(train_df)

3952

In [20]:
tt = load_from_disk("/opt/ml/data/new_train_dataset/train_concat_es_no_dup_wiki")
len(tt)

7904

In [99]:
train_dataset['answers'][0]['answer_start']

array([229], dtype=int32)

In [100]:
train_df.head()

,answers,context,id,question
0,"{'answer_start': [229], 'text': ['하원']}",미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,mrc-1-000067,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?
1,"{'answer_start': [212], 'text': ['《경영의 실제》']}",'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,mrc-0-004397,현대적 인사조직관리의 시발점이 된 책은?
2,"{'answer_start': [503], 'text': ['백성']}",강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,mrc-1-000362,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?
3,"{'answer_start': [615], 'text': ['중국']}","불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",mrc-0-001510,11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?
4,"{'answer_start': [30], 'text': ['4개']}",동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,mrc-0-000823,명문이 적힌 유물을 구성하는 그릇의 총 개수는?


In [75]:
train_dataset = load_from_disk("/opt/ml/data/new_train_dataset/train").to_pandas() # 전처리된 데이터를 테스트로 활용
origin_train_dataset = load_from_disk("/opt/ml/data/new_train_dataset/train_origin_concat_no_dup_wiki").to_pandas()
validation =  load_from_disk("/opt/ml/data/new_train_dataset/validation").to_pandas()

In [76]:
validation['context'][0]

'순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법시험에 합격하여 판사로 임용되었고 대법원 재판연구관, 수원지법 부장판사, 사법연수원 교수, 특허법원 부장판사 등을 거쳐 능력을 인정받았다. 2003년 최종영 대법원장의 지명으로 헌법재판소 재판관을 역임하였다. 경제민주화위원회(위원장 장하성이 소액주주들을 대표해 한보철강 부실대출에 책임이 있는 이철수 전 제일은행장 등 임원 4명을 상대로 제기한 손해배상청구소송에서 서울지방법원 민사합의17부는 1998년 7월 24일에 "한보철강에 부실 대출하여 은행에 막대한 손해를 끼친 점이 인정된다"며 "원고가 배상을 청구한 400억원 전액을 은행에 배상하라"고 하면서 부실 경영인에 대한 최초의 배상 판결을 했다. 2004년 10월 신행정수도의건설을위한특별조치법 위헌 확인 소송에서 9인의 재판관 중 유일하게 각하 견해를 내었다. 소수의견에서 전효숙 재판관은 다수견해의 문제점을 지적하면서 관습헌법 법리를 부정하였다. 전효숙 재판관은 서울대학교 근대법학교육 백주년 기념관에서 열린 강연에서, 국회가 고도의 정치적인 사안을 정치로 풀기보다는 헌법재판소에 무조건 맡겨서 해결하려는 자세는 헌법재판소에게 부담스럽다며 소회를 밝힌 바 있다.'

In [77]:
train_dataset = load_from_disk("/opt/ml/data/new_train_dataset/train_origin_concat_no_duplication").to_pandas()
train_dataset_old = load_from_disk('/opt/ml/data/train_dataset/elastic_train').to_pandas()
valid_dataset = load_from_disk("/opt/ml/data/train_dataset/validation").to_pandas()

In [78]:
valid_dataset['context'][0]

'순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법시험에 합격하여 판사로 임용되었고 대법원 재판연구관, 수원지법 부장판사, 사법연수원 교수, 특허법원 부장판사 등을 거쳐 능력을 인정받았다. 2003년 최종영 대법원장의 지명으로 헌법재판소 재판관을 역임하였다.\\n\\n경제민주화위원회(위원장 장하성이 소액주주들을 대표해 한보철강 부실대출에 책임이 있는 이철수 전 제일은행장 등 임원 4명을 상대로 제기한 손해배상청구소송에서 서울지방법원 민사합의17부는 1998년 7월 24일에 "한보철강에 부실 대출하여 은행에 막대한 손해를 끼친 점이 인정된다"며 "원고가 배상을 청구한 400억원 전액을 은행에 배상하라"고 하면서 부실 경영인에 대한 최초의 배상 판결을 했다. \\n\\n2004년 10월 신행정수도의건설을위한특별조치법 위헌 확인 소송에서 9인의 재판관 중 유일하게 각하 견해를 내었다. 소수의견에서 전효숙 재판관은 다수견해의 문제점을 지적하면서 관습헌법 법리를 부정하였다. 전효숙 재판관은 서울대학교 근대법학교육 백주년 기념관에서 열린 강연에서, 국회가 고도의 정치적인 사안을 정치로 풀기보다는 헌법재판소에 무조건 맡겨서 해결하려는 자세는 헌법재판소에게 부담스럽다며 소회를 밝힌 바 있다.'

In [101]:
train_dataset['context'][2]

'강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전제 독재의 가능성이 보일 수도 있었으나, 스스로 황권을 조절하고 정치의 일부는 재상들이나 대신들과 의논하였으며 당시 궁핍하게 살고 있는 한족들의 사정을 잘 알고 있던 한족 대신들의 의견을 수용하여 정책을 실행하고 선정을 베풀었다. 프랑스의 예수회 선교사 부베는 루이 14세에게 다음과 같이 보고하였다. 강희제는 세상에서 가장 부유한 군주입니다. 그럼에도 황제인 그의 생활용품들은 사치스러움과 화려함과는 거리가 멀다 못해 소박하기 그지없습니다. 역대 제왕들 가운데 전례없는 일입니다. 강희제 스스로도 자신이 직접 쓴 《근검록》에서 다음과 같이 쓰고 있다 모든 비용은 백성들의 피땀으로 얻어진 것이니 주인된 황제로서 절제하고 절제함은 당연한 것이 아닌가 이런 강희제의 인자한 정치는 한족이 만주족의 청나라를 지지하게 만드는 데에 크게 일조하였다. 1717년(강희 56년) 강희제는 〈고별상유〉(告別上諭), 즉 마지막으로 백성들에게 바치는 글을 남겼는데 강희제는 “한 가지 일에 부지런하지 않으면 온 천하에 근심을 끼치고, 한 순간에 부지런하지 않으면 천추만대에 우환거리를 남긴다.”라고 역설하였다. 또한 “제왕이 천하를 다스림에 능력이 있는 자를 가까이 두고, 백성들의 세금을 낮추어 주어야 하며, 백성들의 마음을 하나로 묶고, 위태로움이 생기기 전에 나라를 보호하며, 혼란이 있기 전에 이를 먼저 파악하여 잘 다스리고, 관대하고 엄격함의 조화를 이루어 나라를 위한 계책을 도모해야 한다.”라고 후대의 황제에게도 이를 훈계하였다. 강희제는 황제로서 자식과 같은 백성들에게 이런 당부의 말을 남겨 황제로서의 도리를 다하려 하였다.'

In [84]:
train_dataset_old['context'][2]

'강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전제 독재의 가능성이 보일 수도 있었으나, 스스로 황권을 조절하고 정치의 일부는 재상들이나 대신들과 의논하였으며 당시 궁핍하게 살고 있는 한족들의 사정을 잘 알고 있던 한족 대신들의 의견을 수용하여 정책을 실행하고 선정을 베풀었다. 프랑스의 예수회 선교사 부베는 루이 14세에게 다음과 같이 보고하였다. \\n강희제는 세상에서 가장 부유한 군주입니다. 그럼에도 황제인 그의 생활용품들은 사치스러움과 화려함과는 거리가 멀다 못해 소박하기 그지없습니다. 역대 제왕들 가운데 전례없는 일입니다.\\n강희제 스스로도 자신이 직접 쓴 《근검록》에서 다음과 같이 쓰고 있다\\n모든 비용은 백성들의 피땀으로 얻어진 것이니 주인된 황제로서 절제하고 절제함은 당연한 것이 아닌가\\n\\n이런 강희제의 인자한 정치는 한족이 만주족의 청나라를 지지하게 만드는 데에 크게 일조하였다. 1717년(강희 56년) 강희제는 〈고별상유〉(告別上諭), 즉 마지막으로 백성들에게 바치는 글을 남겼는데 강희제는 “한 가지 일에 부지런하지 않으면 온 천하에 근심을 끼치고, 한 순간에 부지런하지 않으면 천추만대에 우환거리를 남긴다.”라고 역설하였다. 또한 “제왕이 천하를 다스림에 능력이 있는 자를 가까이 두고, 백성들의 세금을 낮추어 주어야 하며, 백성들의 마음을 하나로 묶고, 위태로움이 생기기 전에 나라를 보호하며, 혼란이 있기 전에 이를 먼저 파악하여 잘 다스리고, 관대하고 엄격함의 조화를 이루어 나라를 위한 계책을 도모해야 한다.”라고 후대의 황제에게도 이를 훈계하였다. 강희제는 황제로서 자식과 같은 백성들에게 이런 당부의 말을 남겨 황제로서의 도리를 다하려 하였다. 몽골 초원은 홍타이지 이후 청나라의 지배를 받고 있었으나, 조공을 올리고 자치권을 인정받았다. 이미 1675년(강희 14년) 차하르 부족이 당시 삼번의 난으로 청나라에 생긴 혼란을 틈타 중원으로 쳐들어왔다. 이에 노한 강희제는 몽골을 내몽골과 외몽골로 나누